In [1]:
import sys
import utilities
from sklearn.cluster import DBSCAN
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

In [2]:
convs = utilities.deserialize_embeddings(utilities.read_conversations('all_conversations.csv'))
convs.shape

100%|██████████████████████| 9941/9941 [00:00<00:00, 15048.49it/s]


(9941, 9)

In [3]:
convs_unique = convs.drop_duplicates(subset=['content'])
convs_unique.shape

(2353, 9)

In [4]:
embeddings = np.stack(convs_unique['embeddings'].values)

#embeddings = StandardScaler().fit_transform(embeddings)

embeddings

array([[ 0.04540386, -0.0597158 ,  0.00585281, ..., -0.01651808,
        -0.00719066,  0.02109064],
       [ 0.05800414, -0.02655935, -0.01830715, ..., -0.03803307,
        -0.00067973,  0.04401809],
       [ 0.0294239 , -0.01055937,  0.00114897, ..., -0.0435928 ,
         0.00267964,  0.03531618],
       ...,
       [ 0.03709601, -0.07363608, -0.05168182, ..., -0.0122634 ,
        -0.03967475,  0.04252081],
       [ 0.02128764, -0.04637908, -0.014529  , ...,  0.02498692,
         0.02394539,  0.02344605],
       [ 0.01743094, -0.06102209, -0.00964174, ..., -0.00132433,
         0.00049291,  0.00679866]], shape=(2353, 768))

In [21]:
max_score = 0

def compute_score(noise, clusters):
    if clusters > 200:
        return 0
    return (1-noise) * clusters

def message(s, eps, num_clusters, noise, min_samples):
    print(f"{s}: Epsilon: {eps} Clusters: {num_clusters} Noise%: {noise}, MinSamples: {min_samples}")

for data in [dict(name="normal",data=embeddings), dict(name="scaled", data=StandardScaler().fit_transform(embeddings))]:
    print(f"Data: {data['name']}")
    for min_samples in range(5, 10):
        print(f"Samples: {min_samples}")
        for eps_x_100 in range(1, 100, 1):
            eps = eps_x_100 / 100
            #print(f"Epsilon: {eps}")
    
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)  # Adjust parameters as needed
            
            dbscan.fit(data['data'])
            labels = dbscan.labels_
    
            noise = len([label for label in labels if label == -1]) / embeddings.shape[0]
        
            num_clusters = len(np.unique([label for label in labels if label != -1]))
    
            score = compute_score(noise, num_clusters)
            #if score > 0:
            #    print(score)
            if score > max_score:
                max_score = score
                message('improvement', eps, num_clusters, noise, min_samples)
            elif score < max_score:
                continue


Data: normal
Samples: 5
improvement: Epsilon: 0.08 Clusters: 1 Noise%: 0.9978750531236719, MinSamples: 5
improvement: Epsilon: 0.09 Clusters: 4 Noise%: 0.9902252443688908, MinSamples: 5
improvement: Epsilon: 0.1 Clusters: 5 Noise%: 0.9830004249893752, MinSamples: 5
improvement: Epsilon: 0.11 Clusters: 6 Noise%: 0.9804504887377815, MinSamples: 5
improvement: Epsilon: 0.12 Clusters: 7 Noise%: 0.9753506162345942, MinSamples: 5
improvement: Epsilon: 0.13 Clusters: 7 Noise%: 0.9749256268593285, MinSamples: 5
improvement: Epsilon: 0.14 Clusters: 8 Noise%: 0.9715257118572036, MinSamples: 5
improvement: Epsilon: 0.15 Clusters: 8 Noise%: 0.9694007649808755, MinSamples: 5
improvement: Epsilon: 0.16 Clusters: 8 Noise%: 0.9689757756056099, MinSamples: 5
improvement: Epsilon: 0.17 Clusters: 9 Noise%: 0.9655758606034849, MinSamples: 5
improvement: Epsilon: 0.18 Clusters: 9 Noise%: 0.9626009349766256, MinSamples: 5
improvement: Epsilon: 0.19 Clusters: 10 Noise%: 0.9604759881002974, MinSamples: 5
impr

In [22]:
dbscan = DBSCAN(eps=0.54, min_samples=5)  # Adjust parameters as needed
        
dbscan.fit(embeddings)
labels = dbscan.labels_

In [23]:
cluster = defaultdict(list)

for i, label in enumerate(labels):
   cluster[label].append(convs_unique['content'].iloc[i])
    
for num, contents in cluster.items():
    if num == -1:
        continue
    print(num)
    for c in contents:
        print(f'  {c}')

0
  What is an academic advisor?
  Who is my academic advisor?
  who are my academic advisors?
  what is an academic advisor
  who is my academic advisor
  How do I contact my advisor?
  advisor
  contact advisor
  Yes actually. Do I have an assigned advisor?
  Who is my advisor?
  who is my advisor
  who are my advisors
  Who is my advisor
  Who my advisor
  Who are my advisors?
  who is my academic advisor?
  Who is my academic advisor
  who is my advisor?
  what is an advisor
  Who are my advisors
  what is advising
  Who are my academic advisors
  who my advisr
  How do I find my academic advisor?
  What does an advisor do?
  What is an academic advisor
  help me schedule an appointment with my advisor
  What is my advisor's email
  who are my advisors?
  Who’s my advisor?
  Who’s my advisor
  What is an advisor?
  does the advising office have more information
  can you share more information on advising
  i just want a link to some advising info on this portal
  is there an advis